In [1]:
import os
import json
import pandas as pd
import traceback

In [5]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [ ]:
KEY = os.getenv("OPENAI_API_KEY")
print(KEY)

In [3]:
from langchain.chat_models import ChatOpenAI                       

In [ ]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)
llm

In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [13]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


C:\Users\deven\AppData\Local\Temp\ipykernel_2580\3044570348.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [14]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [16]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [17]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [18]:
file_path=r"D:\Projects\iNeuron\MyMCQGen\data.txt"
file_path

'D:\\Projects\\iNeuron\\MyMCQGen\\data.txt'

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

    print(TEXT)

Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processing, scientific visualization, algorithms and systems to extract or extrapolate knowledge and insights from potentially noisy, structured, or unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is "a concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information scie

In [20]:
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [22]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\deven\AppData\Local\Temp\ipykernel_2580\2654919708.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processing, scientific visualization, algorithms and systems to extract or extrapolate knowledge and insights from potentially noisy, structured, or unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is "a concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information sci

In [23]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:5616
Prompt Tokens:4670
Completion Tokens:946
Total Cost:0.008896999999999999


In [25]:
response

{'text': 'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processing, scientific visualization, algorithms and systems to extract or extrapolate knowledge and insights from potentially noisy, structured, or unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is "a concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and in

In [26]:
quiz1=response.get("review")

In [27]:
json.loads(quiz1)

{'1': {'mcq': 'What is data science?',
  'options': {'a': 'A field focused on extracting knowledge from small datasets',
   'b': 'An interdisciplinary field that uses statistics, scientific computing, and algorithms to extract insights from data',
   'c': 'A field that only focuses on structured data',
   'd': 'A field that does not involve domain knowledge'},
  'correct': 'b'},
 '2': {'mcq': 'What is the role of a data scientist?',
  'options': {'a': 'Create programming code and combine it with historical data',
   'b': 'Analyze data using only statistics',
   'c': 'Extract insights from data using statistical knowledge',
   'd': 'Focus on data visualization only'},
  'correct': 'c'},
 '3': {'mcq': 'How is data science related to statistics?',
  'options': {'a': 'Data science is a completely separate field from statistics',
   'b': 'Statistics is an essential part of data science',
   'c': 'Data science does not involve quantitative data',
   'd': 'Statistics only focuses on qualitati

In [28]:
quiz=json.loads(quiz1)



In [29]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [30]:
quiz_table_data


[{'MCQ': 'What is data science?',
  'Choices': 'a: A field focused on extracting knowledge from small datasets | b: An interdisciplinary field that uses statistics, scientific computing, and algorithms to extract insights from data | c: A field that only focuses on structured data | d: A field that does not involve domain knowledge',
  'Correct': 'b'},
 {'MCQ': 'What is the role of a data scientist?',
  'Choices': 'a: Create programming code and combine it with historical data | b: Analyze data using only statistics | c: Extract insights from data using statistical knowledge | d: Focus on data visualization only',
  'Correct': 'c'},
 {'MCQ': 'How is data science related to statistics?',
  'Choices': 'a: Data science is a completely separate field from statistics | b: Statistics is an essential part of data science | c: Data science does not involve quantitative data | d: Statistics only focuses on qualitative data',
  'Correct': 'b'},
 {'MCQ': 'What is the difference between data scien

In [31]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,What is data science?,a: A field focused on extracting knowledge fro...,b
1,What is the role of a data scientist?,a: Create programming code and combine it with...,c
2,How is data science related to statistics?,a: Data science is a completely separate field...,b
3,What is the difference between data science an...,"a: Data science involves working with smaller,...",c
4,How does cloud computing benefit data science?,a: Cloud computing does not offer access to co...,b


In [ ]:
import os
import json
import pandas as pd
import traceback

In [15]:
KEY = os.getenv("OPENAI_API_KEY")
print(KEY)

None


In [13]:
from langchain.chat_models import ChatOpenAI                

ModuleNotFoundError: No module named 'langchain_community'